# 데이터 소스별 요약 통계

이 노트북은 각 데이터 소스별로 요약 통계를 생성합니다.

## 데이터 소스
1. **Player Match Stats**: `data/player_match_stats/*.csv` (리그별 경기 통계)
2. **VAEP Results**: `data/vaep_results/player_season_vaep_atomic.csv` + `data/wyscout/players.json` (VAEP 데이터와 선수 정보 연동)


## 1. 라이브러리 및 경로 설정


In [44]:
import pandas as pd
import json
from pathlib import Path
import numpy as np

# 파일 경로 설정
stats_dir = Path('../data/player_match_stats')
vaep_file = Path('../data/vaep_results/player_season_vaep_atomic.csv')
player_game_vaep_file = Path('../data/vaep_results/player_game_vaep_atomic.csv')
players_json = Path('../data/wyscout/players.json')
matches_dir = Path('../data/wyscout')
competitions_json = Path('../data/wyscout/competitions.json')

# 리그 파일 목록
league_files = [
    'ENG-Premier League-2017-18.csv',
    'ESP-La Liga-2017-18.csv',
    'FRA-Ligue 1-2017-18.csv',
    'GER-Bundesliga-2017-18.csv',
    'ITA-Serie A-2017-18.csv'
]


## 2. Player Match Stats 요약 통계


In [45]:
print("=" * 80)
print("Player Match Stats 요약 통계")
print("=" * 80)

# 컬럼명 매핑 (merge_premier_league_vaep.ipynb 로직 사용)
column_mapping = {
    # 기본 정보 (이미 올바른 이름)
    'league': 'league',
    'season': 'season',
    'game': 'game',
    'team': 'team',
    'player': 'player',
    'jersey_number': 'jersey_number',
    'nation': 'nation',
    'pos': 'pos',
    'age': 'age',
    # Performance 지표
    'min': 'minutes',
    'Gls': 'goals',
    'Ast': 'assists',
    'PK': 'penalties',
    'PKatt': 'penalty_attempts',
    'Sh': 'shots',
    'SoT': 'shots_on_target',
    'CrdY': 'yellow_cards',
    'CrdR': 'red_cards',
    'Touches': 'touches',
    'Tkl': 'tackles',
    'Int': 'interceptions',
    'Blocks': 'blocks',
    # Expected 지표
    'xG': 'xG',
    'npxG': 'npxG',
    'xAG': 'xAG',
    # SCA/GCA
    'SCA': 'SCA',
    'GCA': 'GCA',
    # Passes
    'Cmp': 'passes_completed',
    'Att': 'passes_attempted',
    'Cmp%': 'pass_completion_pct',
    'PrgP': 'progressive_passes',
    # Carries
    'Carries': 'carries',
    'PrgC': 'progressive_carries',
    # Take-Ons
    'Succ': 'take_ons_successful'
}

all_leagues_data = []
league_summaries = []

for league_file in league_files:
    file_path = stats_dir / league_file
    if not file_path.exists():
        print(f"경고: {league_file} 파일을 찾을 수 없습니다.")
        continue
    
    try:
        # header=[0, 1]로 읽기 (multi-level header)
        df = pd.read_csv(file_path, header=[0, 1])
        
        # 컬럼명 매핑 (multi-level header 처리)
        rename_dict = {}
        col_list = list(df.columns)
        
        for i, col in enumerate(col_list):
            if isinstance(col, tuple):
                # Unnamed 컬럼 처리: 첫 번째 5개 컬럼은 league, season, game, team, player
                if i < 5 and str(col[1]).startswith('Unnamed'):
                    base_cols = ['league', 'season', 'game', 'team', 'player']
                    rename_dict[col] = base_cols[i]
                # 두 번째 레벨 이름 사용 (실제 컬럼명)
                elif col[1] and not str(col[1]).startswith('Unnamed'):
                    actual_name = col[1]
                    # 매핑 딕셔너리에 있으면 사용
                    if actual_name in column_mapping:
                        rename_dict[col] = column_mapping[actual_name]
                    else:
                        rename_dict[col] = actual_name
                # 첫 번째 레벨 이름 사용
                elif col[0] and not str(col[0]).startswith('Unnamed'):
                    if col[0] in column_mapping:
                        rename_dict[col] = column_mapping[col[0]]
                    else:
                        rename_dict[col] = col[0]
                else:
                    # 둘 다 Unnamed인 경우 인덱스 기반 처리
                    rename_dict[col] = f'col_{i}'
            else:
                if col in column_mapping:
                    rename_dict[col] = column_mapping[col]
                else:
                    rename_dict[col] = col
        
        # rename 적용
        df = df.rename(columns=rename_dict)
        
        # Multi-level 컬럼을 단일 레벨로 변환 (rename 후에도 tuple이 남아있을 수 있음)
        new_columns = []
        for col in df.columns:
            if isinstance(col, tuple):
                # rename_dict에 있으면 사용, 없으면 두 번째 레벨 또는 첫 번째 레벨 사용
                if col in rename_dict:
                    new_columns.append(rename_dict[col])
                elif col[1] and not str(col[1]).startswith('Unnamed'):
                    new_columns.append(col[1])
                elif col[0] and not str(col[0]).startswith('Unnamed'):
                    new_columns.append(col[0])
                else:
                    new_columns.append(str(col))
            else:
                new_columns.append(col)
        
        df.columns = new_columns
        
        # Take-Ons의 Att 컬럼 처리 (Passes의 Att와 구분)
        if 'passes_attempted' in df.columns:
            # passes_attempted가 두 번 나타나는 경우 (Passes와 Take-Ons 모두)
            cols_list = df.columns.tolist()
            passes_att_indices = [i for i, col in enumerate(cols_list) if col == 'passes_attempted']
            if len(passes_att_indices) == 2:
                # 두 번째 passes_attempted를 take_ons_attempted로 변경
                cols_list[passes_att_indices[1]] = 'take_ons_attempted'
                df.columns = cols_list
            elif 'Att' in df.columns:
                # Att가 남아있으면 take_ons_attempted로 변경
                df = df.rename(columns={'Att': 'take_ons_attempted'})
        
        # 첫 번째 행이 헤더 정보인 경우 제거 (league 컬럼 값이 'league'인 행)
        if len(df) > 0 and 'league' in df.columns:
            try:
                league_col_idx = df.columns.get_loc('league')
                first_league_val = df.iloc[0, league_col_idx]
                # 단일 값으로 변환
                if hasattr(first_league_val, 'item'):
                    first_league_val = first_league_val.item()
                if isinstance(first_league_val, str) and first_league_val.strip() == 'league':
                    df = df.iloc[1:].reset_index(drop=True)
            except Exception:
                pass
        
        # 수치형 컬럼 변환
        numeric_cols = ['minutes', 'goals', 'assists', 'shots', 'shots_on_target', 
                       'touches', 'tackles', 'interceptions', 'blocks',
                       'xG', 'npxG', 'xAG', 'SCA', 'GCA',
                       'passes_completed', 'passes_attempted', 'pass_completion_pct',
                       'progressive_passes', 'carries', 'progressive_carries',
                       'take_ons_attempted', 'take_ons_successful',
                       'yellow_cards', 'red_cards', 'penalties', 'penalty_attempts']
        
        for col in numeric_cols:
            if col in df.columns:
                try:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                except Exception as e:
                    # 경고만 출력하고 계속 진행
                    pass
        
        league_name = league_file.replace('-2017-18.csv', '')
        
        # 리그별 요약 통계
        summary = {
            '리그': league_name,
            '총 경기 수': df['game'].nunique() if 'game' in df.columns else 0,
            '총 선수 수': df['player'].nunique() if 'player' in df.columns else 0,
            '총 팀 수': df['team'].nunique() if 'team' in df.columns else 0,
            '총 레코드 수': len(df)
        }
        
        if 'game' in df.columns:
            game_count = df['game'].nunique()
            if game_count > 0:
                summary['평균 경기당 선수 수'] = len(df) / game_count
        
        if 'minutes' in df.columns:
            summary['총 경기 시간'] = df['minutes'].sum()
            summary['평균 경기 시간'] = df['minutes'].mean()
        if 'goals' in df.columns:
            summary['총 골 수'] = df['goals'].sum()
            summary['평균 골 수'] = df['goals'].mean()
        if 'assists' in df.columns:
            summary['총 어시스트 수'] = df['assists'].sum()
            summary['평균 어시스트 수'] = df['assists'].mean()
        
        league_summaries.append(summary)
        all_leagues_data.append(df)
        
        print(f"\n{league_name}:")
        print(f"  총 경기 수: {summary['총 경기 수']}")
        print(f"  총 선수 수: {summary['총 선수 수']}")
        print(f"  총 팀 수: {summary['총 팀 수']}")
        print(f"  총 레코드 수: {summary['총 레코드 수']:,}")
        
    except Exception as e:
        print(f"오류: {league_file} 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

# 전체 통합 요약
if all_leagues_data:
    all_stats_df = pd.concat(all_leagues_data, ignore_index=True)
    
    print("\n" + "=" * 80)
    print("전체 통합 요약")
    print("=" * 80)
    print(f"총 리그 수: {len(all_leagues_data)}")
    print(f"총 경기 수: {all_stats_df['game'].nunique() if 'game' in all_stats_df.columns else 0}")
    print(f"총 선수 수: {all_stats_df['player'].nunique() if 'player' in all_stats_df.columns else 0}")
    print(f"총 팀 수: {all_stats_df['team'].nunique() if 'team' in all_stats_df.columns else 0}")
    print(f"총 레코드 수: {len(all_stats_df):,}")
    
    if 'minutes' in all_stats_df.columns:
        print(f"총 경기 시간: {all_stats_df['minutes'].sum():,.0f}분")
        print(f"평균 경기 시간: {all_stats_df['minutes'].mean():.1f}분")
    
    if 'goals' in all_stats_df.columns:
        print(f"총 골 수: {all_stats_df['goals'].sum():,.0f}")
        print(f"평균 골 수: {all_stats_df['goals'].mean():.2f}")
    
    if 'assists' in all_stats_df.columns:
        print(f"총 어시스트 수: {all_stats_df['assists'].sum():,.0f}")
        print(f"평균 어시스트 수: {all_stats_df['assists'].mean():.2f}")
    
    # 리그별 요약 테이블
    summary_df = pd.DataFrame(league_summaries)
    print("\n" + "=" * 80)
    print("리그별 요약 테이블")
    print("=" * 80)
    print(summary_df.to_string(index=False))
    
    # 포지션별 통계
    if 'pos' in all_stats_df.columns:
        print("\n" + "=" * 80)
        print("포지션별 선수 수")
        print("=" * 80)
        pos_counts = all_stats_df['pos'].value_counts()
        print(pos_counts)
    
    # 국적별 통계
    if 'nation' in all_stats_df.columns:
        print("\n" + "=" * 80)
        print("국적별 상위 10개")
        print("=" * 80)
        nation_counts = all_stats_df['nation'].value_counts().head(10)
        print(nation_counts)


Player Match Stats 요약 통계

ENG-Premier League:
  총 경기 수: 380
  총 선수 수: 515
  총 팀 수: 20
  총 레코드 수: 10,448

ESP-La Liga:
  총 경기 수: 380
  총 선수 수: 555
  총 팀 수: 20
  총 레코드 수: 10,556

FRA-Ligue 1:
  총 경기 수: 382
  총 선수 수: 559
  총 팀 수: 21
  총 레코드 수: 10,576

GER-Bundesliga:
  총 경기 수: 308
  총 선수 수: 473
  총 팀 수: 19
  총 레코드 수: 8,512

ITA-Serie A:
  총 경기 수: 380
  총 선수 수: 533
  총 팀 수: 20
  총 레코드 수: 10,589

전체 통합 요약
총 리그 수: 5
총 경기 수: 1830
총 선수 수: 2580
총 팀 수: 100
총 레코드 수: 50,681
총 경기 시간: 3,610,396분
평균 경기 시간: 71.2분
총 골 수: 4,800
평균 골 수: 0.09
총 어시스트 수: 3,411
평균 어시스트 수: 0.07

리그별 요약 테이블
                리그  총 경기 수  총 선수 수  총 팀 수  총 레코드 수  평균 경기당 선수 수  총 경기 시간  평균 경기 시간  총 골 수   평균 골 수  총 어시스트 수  평균 어시스트 수
ENG-Premier League     380     515     20    10448    27.494737 751193.0 71.898258  988.0 0.094564     728.0   0.069678
       ESP-La Liga     380     555     20    10556    27.778947 750869.0 71.131963  994.0 0.094164     720.0   0.068208
       FRA-Ligue 1     382     559     21    10576    27.685864 753

## 3. VAEP Results + Players.json 연동 데이터 요약 통계


In [46]:
print("=" * 80)
print("VAEP Results + Players.json 연동 데이터 요약 통계")
print("=" * 80)

# 1. VAEP 데이터 로드
print("\n1. VAEP 데이터 로드 중...")
vaep_df = pd.read_csv(vaep_file)

# 컬럼명 정규화
column_mapping_vaep = {
    'player_id': 'playerId',
    'season_vaep_total': 'season_vaep_total',
    'avg_vaep_per_game': 'season_vaep_per_match',
    'total_actions': 'num_events',
    'num_games': 'matches_played',
    'vaep_per90': 'season_vaep_per90_avg'
}

for old_col, new_col in column_mapping_vaep.items():
    if old_col in vaep_df.columns:
        vaep_df = vaep_df.rename(columns={old_col: new_col})

print(f"VAEP 데이터 로드 완료: {len(vaep_df):,} 행")
print(f"컬럼: {list(vaep_df.columns)}")

# playerId=0 제외
vaep_df = vaep_df[vaep_df['playerId'] != 0].copy()
print(f"playerId=0 제외 후: {len(vaep_df):,} 행")

# 1-1. VAEP 데이터에 리그 정보 추가 (merge_premier_league_vaep.ipynb 로직 사용)
print("\n1-1. VAEP 데이터에 리그 정보 추가 중...")

# competitions.json에서 리그 ID -> 이름 매핑 생성
with open(competitions_json, 'r', encoding='utf-8') as f:
    competitions = json.load(f)

# 리그 이름 매핑 (competitions.json -> 경기 스탯 형식)
league_name_mapping = {
    'English first division': 'ENG-Premier League',
    'Spanish first division': 'ESP-La Liga',
    'Italian first division': 'ITA-Serie A',
    'French first division': 'FRA-Ligue 1',
    'German first division': 'GER-Bundesliga',
    'European Championship': 'European Championship',
    'World Cup': 'World Cup'
}

competition_id_to_name = {}
for comp in competitions:
    original_name = comp.get('name', f"Competition {comp['wyId']}")
    mapped_name = league_name_mapping.get(original_name, original_name)
    competition_id_to_name[comp['wyId']] = mapped_name

# matches 파일들에서 game_id -> competitionId 매핑 생성
game_id_to_competition = {}
matches_files = [
    'matches_England.json', 'matches_Spain.json', 'matches_Italy.json',
    'matches_Germany.json', 'matches_France.json',
    'matches_European_Championship.json', 'matches_World_Cup.json'
]

for matches_file in matches_files:
    matches_path = matches_dir / matches_file
    if matches_path.exists():
        with open(matches_path, 'r', encoding='utf-8') as f:
            matches = json.load(f)
        for match in matches:
            game_id = match.get('wyId')
            competition_id = match.get('competitionId')
            if game_id and competition_id:
                game_id_to_competition[int(game_id)] = int(competition_id)

print(f"게임 ID -> 리그 매핑: {len(game_id_to_competition):,}개")

# player_game_vaep_atomic.csv에서 player_id별 game_id 추출
print("player_game_vaep_atomic.csv 로드 중...")
player_game_df = pd.read_csv(player_game_vaep_file)
print(f"게임별 VAEP 데이터: {len(player_game_df):,} 행")

# player_id별로 가장 많이 나타난 리그 찾기
player_league_counts = {}
for _, row in player_game_df.iterrows():
    player_id = int(row['player_id'])
    game_id = int(row['game_id'])
    
    if game_id in game_id_to_competition:
        competition_id = game_id_to_competition[game_id]
        league_name = competition_id_to_name.get(competition_id, f"Unknown {competition_id}")
        
        if player_id not in player_league_counts:
            player_league_counts[player_id] = {}
        if league_name not in player_league_counts[player_id]:
            player_league_counts[player_id][league_name] = 0
        player_league_counts[player_id][league_name] += 1

# 각 선수별로 가장 많이 나타난 리그 선택
player_id_to_league = {}
for player_id, league_counts in player_league_counts.items():
    most_common_league = max(league_counts.items(), key=lambda x: x[1])[0]
    player_id_to_league[player_id] = most_common_league

print(f"선수별 리그 정보: {len(player_id_to_league):,}명")

# vaep_df에 리그 정보 추가
vaep_df['league'] = vaep_df['playerId'].map(player_id_to_league)
print(f"VAEP 데이터에 리그 정보 추가: {vaep_df['league'].notna().sum()}명 / {len(vaep_df)}명")

# 월드컵, 유로 챔피언스 리그 데이터 제외
excluded_leagues = ['World Cup', 'European Championship']
before_filter = len(vaep_df)
vaep_df_filtered = vaep_df[~vaep_df['league'].isin(excluded_leagues)].copy()
after_filter = len(vaep_df_filtered)
print(f"월드컵/유로 챔피언스 리그 제외: {before_filter - after_filter}명 제외, 남은 데이터: {after_filter}명")
print(f"제외된 리그별 선수 수:")
excluded_data = vaep_df[vaep_df['league'].isin(excluded_leagues)]
if len(excluded_data) > 0:
    print(excluded_data['league'].value_counts())

# VAEP 데이터 기본 통계 (월드컵/유로 제외)
print("\n" + "=" * 80)
print("VAEP 데이터 기본 통계 (월드컵/유로 챔피언스 제외)")
print("=" * 80)
print(f"총 선수 수: {vaep_df_filtered['playerId'].nunique():,}")
print(f"총 레코드 수: {len(vaep_df_filtered):,}")

# 리그별 선수 수 (월드컵/유로 제외)
print("\n" + "=" * 80)
print("VAEP 데이터 리그별 선수 수 (월드컵/유로 제외)")
print("=" * 80)
if 'league' in vaep_df_filtered.columns:
    vaep_league_counts = vaep_df_filtered['league'].value_counts().sort_index()
    for league, count in vaep_league_counts.items():
        print(f"  {league}: {count:,}명")

if 'season_vaep_total' in vaep_df_filtered.columns:
    print(f"\n시즌 VAEP 총합:")
    print(f"  평균: {vaep_df_filtered['season_vaep_total'].mean():.4f}")
    print(f"  중앙값: {vaep_df_filtered['season_vaep_total'].median():.4f}")
    print(f"  최대값: {vaep_df_filtered['season_vaep_total'].max():.4f}")
    print(f"  최소값: {vaep_df_filtered['season_vaep_total'].min():.4f}")

if 'season_vaep_per90_avg' in vaep_df_filtered.columns:
    print(f"\n경기당 VAEP (per90):")
    print(f"  평균: {vaep_df_filtered['season_vaep_per90_avg'].mean():.4f}")
    print(f"  중앙값: {vaep_df_filtered['season_vaep_per90_avg'].median():.4f}")
    print(f"  최대값: {vaep_df_filtered['season_vaep_per90_avg'].max():.4f}")
    print(f"  최소값: {vaep_df_filtered['season_vaep_per90_avg'].min():.4f}")

if 'matches_played' in vaep_df_filtered.columns:
    print(f"\n경기 수:")
    print(f"  평균: {vaep_df_filtered['matches_played'].mean():.2f}")
    print(f"  중앙값: {vaep_df_filtered['matches_played'].median():.2f}")
    print(f"  최대값: {vaep_df_filtered['matches_played'].max():.0f}")
    print(f"  최소값: {vaep_df_filtered['matches_played'].min():.0f}")

if 'num_events' in vaep_df_filtered.columns:
    print(f"\n이벤트 수:")
    print(f"  평균: {vaep_df_filtered['num_events'].mean():.2f}")
    print(f"  중앙값: {vaep_df_filtered['num_events'].median():.2f}")
    print(f"  최대값: {vaep_df_filtered['num_events'].max():.0f}")
    print(f"  최소값: {vaep_df_filtered['num_events'].min():.0f}")


VAEP Results + Players.json 연동 데이터 요약 통계

1. VAEP 데이터 로드 중...
VAEP 데이터 로드 완료: 3,031 행
컬럼: ['playerId', 'season_vaep_total', 'season_vaep_per_match', 'num_events', 'matches_played', 'season_vaep_per90_avg']
playerId=0 제외 후: 3,030 행

1-1. VAEP 데이터에 리그 정보 추가 중...
게임 ID -> 리그 매핑: 1,941개
player_game_vaep_atomic.csv 로드 중...
게임별 VAEP 데이터: 55,369 행
선수별 리그 정보: 3,031명
VAEP 데이터에 리그 정보 추가: 3030명 / 3030명
월드컵/유로 챔피언스 리그 제외: 478명 제외, 남은 데이터: 2552명
제외된 리그별 선수 수:
World Cup                272
European Championship    206
Name: league, dtype: int64

VAEP 데이터 기본 통계 (월드컵/유로 챔피언스 제외)
총 선수 수: 2,552
총 레코드 수: 2,552

VAEP 데이터 리그별 선수 수 (월드컵/유로 제외)
  ENG-Premier League: 495명
  ESP-La Liga: 548명
  FRA-Ligue 1: 524명
  GER-Bundesliga: 462명
  ITA-Serie A: 523명

시즌 VAEP 총합:
  평균: 11.1168
  중앙값: 9.8150
  최대값: 54.6784
  최소값: 0.0058

경기당 VAEP (per90):
  평균: 0.4782
  중앙값: 0.4799
  최대값: 1.3754
  최소값: 0.0058

경기 수:
  평균: 20.45
  중앙값: 22.00
  최대값: 49
  최소값: 1

이벤트 수:
  평균: 1666.51
  중앙값: 1423.00
  최대값: 8510
  최소값: 1


In [47]:
# 2. Players.json 로드 및 매핑
print("\n" + "=" * 80)
print("Players.json 로드 및 연동")
print("=" * 80)

print("\n2. Players.json 로드 중...")
with open(players_json, 'r', encoding='utf-8') as f:
    players_data = json.load(f)

print(f"Players.json 로드 완료: {len(players_data):,}명")

# 선수 정보 매핑 생성
player_id_to_name = {}
player_id_to_nation = {}
player_id_to_position = {}
player_id_to_birth_date = {}

for player in players_data:
    player_id = player.get('wyId')
    if player_id:
        player_id = int(player_id)
        
        # 이름
        if 'shortName' in player:
            player_id_to_name[player_id] = player['shortName']
        elif 'firstName' in player and 'lastName' in player:
            player_id_to_name[player_id] = f"{player['firstName']} {player['lastName']}"
        
        # 국적
        if 'passportArea' in player and 'name' in player['passportArea']:
            player_id_to_nation[player_id] = player['passportArea']['name']
        
        # 포지션 (role)
        if 'role' in player:
            role = player['role']
            if 'code2' in role:
                player_id_to_position[player_id] = role['code2']
            elif 'code3' in role:
                player_id_to_position[player_id] = role['code3']
        
        # 생년월일
        if 'birthDate' in player:
            player_id_to_birth_date[player_id] = player['birthDate']

print(f"\n매핑 생성 완료:")
print(f"  ID -> 이름: {len(player_id_to_name):,}명")
print(f"  ID -> 국적: {len(player_id_to_nation):,}명")
print(f"  ID -> 포지션: {len(player_id_to_position):,}명")
print(f"  ID -> 생년월일: {len(player_id_to_birth_date):,}명")



Players.json 로드 및 연동

2. Players.json 로드 중...
Players.json 로드 완료: 3,603명

매핑 생성 완료:
  ID -> 이름: 3,603명
  ID -> 국적: 3,603명
  ID -> 포지션: 3,603명
  ID -> 생년월일: 3,603명


In [48]:
# 3. VAEP 데이터와 Players.json 연동 (월드컵/유로 제외된 데이터 사용)
print("\n" + "=" * 80)
print("VAEP + Players.json 연동 데이터 (월드컵/유로 챔피언스 제외)")
print("=" * 80)

# VAEP 데이터에 선수 정보 추가 (필터링된 데이터 사용)
vaep_df_filtered['player_name'] = vaep_df_filtered['playerId'].map(player_id_to_name)
vaep_df_filtered['nation'] = vaep_df_filtered['playerId'].map(player_id_to_nation)
vaep_df_filtered['position'] = vaep_df_filtered['playerId'].map(player_id_to_position)
vaep_df_filtered['birth_date'] = vaep_df_filtered['playerId'].map(player_id_to_birth_date)

print(f"\n연동 결과:")
print(f"  이름 정보 있는 선수: {vaep_df_filtered['player_name'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['player_name'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")
print(f"  국적 정보 있는 선수: {vaep_df_filtered['nation'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['nation'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")
print(f"  포지션 정보 있는 선수: {vaep_df_filtered['position'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['position'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")
print(f"  생년월일 정보 있는 선수: {vaep_df_filtered['birth_date'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['birth_date'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")

# 연동된 데이터 요약 통계
print("\n" + "=" * 80)
print("연동 데이터 요약 통계")
print("=" * 80)

# 국적별 통계
if 'nation' in vaep_df_filtered.columns:
    print("\n국적별 선수 수 (상위 15개):")
    nation_counts = vaep_df_filtered['nation'].value_counts().head(15)
    for nation, count in nation_counts.items():
        print(f"  {nation}: {count:,}명")
    
    print("\n국적별 VAEP 통계 (상위 10개):")
    if 'season_vaep_per90_avg' in vaep_df_filtered.columns:
        nation_vaep = vaep_df_filtered.groupby('nation')['season_vaep_per90_avg'].agg(['mean', 'count']).sort_values('mean', ascending=False).head(10)
        print(nation_vaep)

# 포지션별 통계
if 'position' in vaep_df_filtered.columns:
    print("\n" + "=" * 80)
    print("포지션별 통계")
    print("=" * 80)
    
    pos_counts = vaep_df_filtered['position'].value_counts()
    print("\n포지션별 선수 수:")
    for pos, count in pos_counts.items():
        print(f"  {pos}: {count:,}명")
    
    if 'season_vaep_per90_avg' in vaep_df_filtered.columns:
        print("\n포지션별 평균 VAEP (per90):")
        pos_vaep = vaep_df_filtered.groupby('position')['season_vaep_per90_avg'].agg(['mean', 'count']).sort_values('mean', ascending=False)
        print(pos_vaep)

# 상위 선수 (VAEP 기준)
if 'season_vaep_per90_avg' in vaep_df_filtered.columns and 'player_name' in vaep_df_filtered.columns:
    print("\n" + "=" * 80)
    print("상위 선수 (VAEP per90 기준, 상위 20명)")
    print("=" * 80)
    
    top_players = vaep_df_filtered.nlargest(20, 'season_vaep_per90_avg')[['player_name', 'nation', 'position', 
                                                                  'season_vaep_per90_avg', 'season_vaep_total', 
                                                                  'matches_played', 'num_events']]
    print(top_players.to_string(index=False))

# 경기 수별 분포
if 'matches_played' in vaep_df_filtered.columns:
    print("\n" + "=" * 80)
    print("경기 수별 분포")
    print("=" * 80)
    
    print("\n경기 수 구간별 선수 수:")
    bins = [0, 5, 10, 20, 30, 40, 50, 100, float('inf')]
    labels = ['0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-100', '100+']
    vaep_df_filtered['games_range'] = pd.cut(vaep_df_filtered['matches_played'], bins=bins, labels=labels, right=False)
    games_dist = vaep_df_filtered['games_range'].value_counts().sort_index()
    for range_label, count in games_dist.items():
        print(f"  {range_label}경기: {count:,}명")



VAEP + Players.json 연동 데이터 (월드컵/유로 챔피언스 제외)

연동 결과:
  이름 정보 있는 선수: 2,552 / 2,552 (100.0%)
  국적 정보 있는 선수: 2,552 / 2,552 (100.0%)
  포지션 정보 있는 선수: 2,552 / 2,552 (100.0%)
  생년월일 정보 있는 선수: 2,552 / 2,552 (100.0%)

연동 데이터 요약 통계

국적별 선수 수 (상위 15개):
  Spain: 412명
  Italy: 341명
  France: 244명
  Germany: 197명
  England: 130명
  Brazil: 79명
  Portugal: 51명
  Senegal: 50명
  Croatia: 40명
  Serbia: 40명
  Congo DR: 36명
  C\u00f4te d'Ivoire: 35명
  Ghana: 34명
  Poland: 34명
  Belgium: 32명

국적별 VAEP 통계 (상위 10개):
                              mean  count
nation                                   
Estonia                   0.863316      1
Guinea-Bissau             0.740248      1
Chile                     0.703719      9
Indonesia                 0.703020      1
Armenia                   0.652639      1
Burundi                   0.650677      1
Grenada                   0.644169      1
Guyana                    0.641201      2
Central African Republic  0.630202      3
Netherlands               0.620272     2

## 4. 요약


In [49]:
print("=" * 80)
print("전체 요약 및 리그별 비교")
print("=" * 80)

print("\n1. Player Match Stats:")
if 'all_stats_df' in globals():
    print(f"   - 총 리그 수: {len(all_leagues_data)}")
    print(f"   - 총 경기 수: {all_stats_df['game'].nunique() if 'game' in all_stats_df.columns else 0}")
    print(f"   - 총 선수 수: {all_stats_df['player'].nunique() if 'player' in all_stats_df.columns else 0}")
    print(f"   - 총 레코드 수: {len(all_stats_df):,}")
    
    # Player Match Stats 리그별 선수 수
    if 'league' in all_stats_df.columns:
        print("\n   리그별 선수 수:")
        stats_league_counts = all_stats_df.groupby('league')['player'].nunique().sort_index()
        for league, count in stats_league_counts.items():
            print(f"     {league}: {count:,}명")
else:
    print("   - 데이터가 로드되지 않았습니다.")

print("\n2. VAEP Results + Players.json (월드컵/유로 제외):")
print(f"   - 총 선수 수: {vaep_df_filtered['playerId'].nunique():,}")
print(f"   - 총 레코드 수: {len(vaep_df_filtered):,}")
print(f"   - 이름 정보 연동: {vaep_df_filtered['player_name'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['player_name'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")
print(f"   - 국적 정보 연동: {vaep_df_filtered['nation'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['nation'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")
print(f"   - 포지션 정보 연동: {vaep_df_filtered['position'].notna().sum():,} / {len(vaep_df_filtered):,} ({vaep_df_filtered['position'].notna().sum()/len(vaep_df_filtered)*100:.1f}%)")

# VAEP 리그별 선수 수
if 'league' in vaep_df_filtered.columns:
    print("\n   리그별 선수 수 (월드컵/유로 제외):")
    vaep_league_counts = vaep_df_filtered['league'].value_counts().sort_index()
    for league, count in vaep_league_counts.items():
        print(f"     {league}: {count:,}명")

# 리그별 비교 테이블
print("\n" + "=" * 80)
print("리그별 선수 수 비교 (Player Match Stats vs VAEP)")
print("=" * 80)

if 'all_stats_df' in globals() and 'league' in all_stats_df.columns and 'league' in vaep_df_filtered.columns:
    # Player Match Stats 리그별 선수 수
    stats_league_counts = all_stats_df.groupby('league')['player'].nunique()
    
    # VAEP 리그별 선수 수
    vaep_league_counts = vaep_df_filtered['league'].value_counts()
    
    # 비교 테이블 생성
    comparison_data = []
    all_leagues = sorted(set(list(stats_league_counts.index) + list(vaep_league_counts.index)))
    
    for league in all_leagues:
        stats_count = stats_league_counts.get(league, 0)
        vaep_count = vaep_league_counts.get(league, 0)
        diff = vaep_count - stats_count
        diff_pct = (diff / stats_count * 100) if stats_count > 0 else 0
        comparison_data.append({
            '리그': league,
            'Player Match Stats': stats_count,
            'VAEP (월드컵/유로 제외)': vaep_count,
            '차이': diff,
            '차이 (%)': f"{diff_pct:+.1f}%"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    print("\n" + "=" * 80)
    print("요약:")
    print("=" * 80)
    print(f"Player Match Stats 총 선수 수: {stats_league_counts.sum():,}명")
    print(f"VAEP 총 선수 수 (월드컵/유로 제외): {vaep_league_counts.sum():,}명")
    print(f"차이: {vaep_league_counts.sum() - stats_league_counts.sum():+,}명")

print("\n" + "=" * 80)


전체 요약 및 리그별 비교

1. Player Match Stats:
   - 총 리그 수: 5
   - 총 경기 수: 1830
   - 총 선수 수: 2580
   - 총 레코드 수: 50,681

   리그별 선수 수:
     ENG-Premier League: 515명
     ESP-La Liga: 555명
     FRA-Ligue 1: 559명
     GER-Bundesliga: 473명
     ITA-Serie A: 533명

2. VAEP Results + Players.json (월드컵/유로 제외):
   - 총 선수 수: 2,552
   - 총 레코드 수: 2,552
   - 이름 정보 연동: 2,552 / 2,552 (100.0%)
   - 국적 정보 연동: 2,552 / 2,552 (100.0%)
   - 포지션 정보 연동: 2,552 / 2,552 (100.0%)

   리그별 선수 수 (월드컵/유로 제외):
     ENG-Premier League: 495명
     ESP-La Liga: 548명
     FRA-Ligue 1: 524명
     GER-Bundesliga: 462명
     ITA-Serie A: 523명

리그별 선수 수 비교 (Player Match Stats vs VAEP)
                리그  Player Match Stats  VAEP (월드컵/유로 제외)  차이 차이 (%)
ENG-Premier League                 515               495 -20  -3.9%
       ESP-La Liga                 555               548  -7  -1.3%
       FRA-Ligue 1                 559               524 -35  -6.3%
    GER-Bundesliga                 473               462 -11  -2.3%
       ITA-Serie A